In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Wave_X = c1_array_float[9:]
Wave_Y = c11_array_float[9:]
Ini = Wave_X[0]
end = Wave_X[-1]

# Frequencies = [40.e3]
# F_High_range = [50.e3,100.e3,150.e3,200.e3,300.e3,350.e3]
# F_Low_range = [89.e3, 91.e3, 85.e3, 95.e3, 1.e3, 100.e3]
main_paras = [float(35e3), 3, 4] # main_ paras: [smoth order, yf_padded[peaks]/?, ?*2*np.std(Choosen_points)]
y_min1 = -2.5
y_max1 = 2.5
y_min2 = -80
y_max2 = 80

In [ ]:
# do not change anything in this box. produces igore resulrs
sampling_rate = 1e-6 # s or 0.25 micro seconds
fs = 1 / sampling_rate  # unit is Hz
N1 = end - Ini + 1 #60 #99-40+1=60 # Number of sample points or # of points used in fft (no unit)
N2 = 2 * len(Wave_X) #120 # 160-40=120
T = fs / N1 # sample spacing or frequency resolution, unit is Hz
T_padded = fs / N2

In [ ]:
############## extra info.
f_max = (fs / 2) # unit is Hz
nyquist_freq = 0.5 * (1 / sampling_rate)  # nyquist_freq= 0.5 * sampling frequency (unit is Hz)
#print 'Nyquist frequency =', nyquist_freq / float(1e+6), '(MHz)'

In [ ]:
Y_Smoothed = butter_lowpass_filter(Wave_Y,main_paras[0],fs,2)
# Y_Smoothed = savgol_filter(Wave_Y, main_paras[0], 2) # 91,1(array, window size, polynomial order), cut part of the fft
#results and shows that part that is inside the window

Y_Subtract = Wave_Y - Y_Smoothed  #len(Wave_Y)=#len(Y_Smoothed)=len(Y_Subtract)=len(Wave_X)=60
window = signal.hanning(len(Wave_X))
y_tapered = window * Y_Subtract
padded_y = np.pad(y_tapered, (0,len(Wave_X)), ‘constant’) #(0,60)
New_x = np.linspace(Ini, end+len(Wave_X), 2*len(Wave_X)) #(40,99+60),(2*60)
yf = 2./N1*np.abs(fftshift(fft(Y_Subtract))) #N1=60
xf = np.arange(-fs/2., fs/2., T) # (unit is Hz)
yf_tappered = 2./N1*np.abs(fftshift(fft(y_tapered)))
yf_padded = 2./N2*np.abs(fftshift(fft(padded_y)))
xf_padded = np.arange(-fs/2., fs/2., T_padded)

In [ ]:
Fig1 = plt.figure(figsize=(14, 10))
plt.subplot(2, 2, 1)
plt.plot(Wave_X, Wave_Y, 'k', label='Actual data')
plt.plot(Wave_X, Y_Smoothed,'tab:red', linewidth=2, label='Smoothed data')
plt.plot(Wave_X, Y_Subtract, linewidth=2, label='Subtracted data')
plt.plot(Wave_X, y_tapered, linewidth=3, label='Hanning data')
plt.plot(New_x, padded_y, linewidth=2, label='Padded data')
plt.xlabel('Phase Delay ($\mu$s)', fontsize=16)
# plt.ylabel(‘Delay (ns)‘,fontsize=16)
#plt.title(Name_Plot,fontweight=“bold”)
#plt.title(Date+‘:’+ Name_Plot)#,fontweight=“bold”)
plt.legend()
plt.grid()
plt.subplot(2, 2, 2)
# plt.plot(xf / 1000.0, yf, 'o-', label='Subtracted data')
# plt.plot(xf / 1000.0, yf_tappered, 'o-', label='Tapper data')
plt.plot(xf_padded / 1000.0, yf_padded, 'o-', label='Padded data')
plt.xlabel('Frequency (kHz)', fontsize=16)
plt.ylabel('FFT', fontsize=16)
plt.grid(b=True, which='both')
# plt.ylim(0,0.5)
plt.minorticks_on()
plt.legend()

In [ ]:
peaks, extra = find_peaks(yf_padded)
main_paras = [float(35e3), 1.5, 4]
# print max(yf_padded[peaks]/main_paras[1]) #3
Choosen_points = [i for i in yf_padded if i < max(yf_padded[peaks] / main_paras[1])]
peak_min_hieght = main_paras[2] * np.std(Choosen_points)
peak_max_hieght = 10
peaks2, extra2 = find_peaks(yf_padded, height=(peak_min_hieght, peak_max_hieght ))
#mean of BL, std of BL, max peak, std of BL/max peak, max peak/mean of BL
# std_related =np.round([np.mean(Choosen_points),np.std(Choosen_points),max(yf_padded[peaks]),
#                        np.std(Choosen_points)/max(yf_padded[peaks]),
#                        max(yf_padded[peaks])/np.mean(Choosen_points)],3)
std_related =np.round([np.mean(yf_padded), np.std(yf_padded), max(yf_padded[peaks]), np.std(yf_padded)/ max(yf_padded[peaks]), 
                       max(yf_padded[peaks]) / np.mean(yf_padded)], 3)
print('std_related:', [std_related[0], std_related[1], std_related[2], std_related[3], std_related[4], 
                       main_paras[0], main_paras[1]])
print('******************************')
print('mean of BL:', std_related[0])
print('std of BL:', std_related[1])
print('max peak:', std_related[2], 'max peak/3:', std_related[2] / main_paras[1])
print('std of BL/max peak:', std_related[3])
print('max peak/mean of BL:', std_related[4])
print('******************************')

freq= []
for num in xf_padded[peaks2]:
    if num >= 0:
        freq.append(np.round(num / 1000, 2))
print('freq are:', freq)

height = []
for i in range((len(yf_padded[peaks2]) / 2), len(yf_padded[peaks2])):
      height.append(np.round(yf_padded[peaks2][i], 3))
print('height are:', height)

Name = 'C11'
figname = Name + '.png'
plt.savefig(“/Users/Somayeh/Dropbox/PDF_Results/SKT2_PDF/“+figname,bbox_inches=‘tight’,dpi=400)